In [2]:
!pip install vllm

  Using cached vllm-0.6.4.post1-cp38-abi3-manylinux1_x86_64.whl.metadata (10 kB)
  Using cached sentencepiece-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached pillow-11.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Using cached prometheus_client-0.21.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached prometheus_fastapi_instrumentator-7.0.0-py3-none-any.whl.metadata (13 kB)
  Using cached lm_format_enforcer-0.10.9-py3-none-any.whl.metadata (17 kB)
  Using cached outlines-0.0.46-py3-none-any.whl.metadata (15 kB)
  Using cached partial_json_parser-0.2.1.1.post4-py3-none-any.whl.metadata (6.2 kB)
  Using cached gguf-0.10.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached einops-0.8.0-py3-none-any.whl.metadata (12 kB)
  Using cached compressed_tensors-0.

In [3]:
import vllm
from vllm import LLM, SamplingParams
from typing import List
from vllm.outputs import RequestOutput

/home/eardream2/miniconda3/envs/single/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-21 19:17:00,659	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [4]:
base_model= 'junn991/llama3-8b-1gpu-couple' # SFT모델이 여기에 들어가야함 => 저장된 SFT 절대경로로 지정
gpu_num= 1
max_token: int = 2048

In [5]:
model = LLM(model=base_model, tensor_parallel_size=gpu_num, max_model_len=max_token, gpu_memory_utilization=0.8)

INFO 11-21 19:17:38 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
INFO 11-21 19:17:38 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='junn991/llama3-8b-1gpu-couple', speculative_config=None, tokenizer='junn991/llama3-8b-1gpu-couple', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=junn991/llama3-8b-1g

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:05<00:15,  5.20s/it]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:12<00:12,  6.49s/it]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:15<00:04,  4.88s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:26<00:00,  7.30s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:26<00:00,  6.64s/it]



INFO 11-21 19:20:10 model_runner.py:1077] Loading model weights took 14.9595 GB
INFO 11-21 19:20:11 worker.py:232] Memory profiling results: total_gpu_memory=21.96GiB initial_memory_usage=15.27GiB peak_torch_memory=16.15GiB memory_usage_post_profile=15.30GiB non_torch_memory=0.34GiB kv_cache_size=1.09GiB gpu_memory_utilization=0.80
INFO 11-21 19:20:12 gpu_executor.py:113] # GPU blocks: 557, # CPU blocks: 2048
INFO 11-21 19:20:12 gpu_executor.py:117] Maximum concurrency for 2048 tokens per request: 4.35x
INFO 11-21 19:20:14 model_runner.py:1400] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-21 19:20:14 model_runner.py:1404] If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilization` or switching to eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 

In [ ]:
sampling_params = SamplingParams(top_k=5, top_p=1, max_tokens=max_token, temperature=0.5, best_of=13, n=6,
                                     stop=['<|endoftext|>', '</s>', '<|im_end|>','<|end_of_text|>'])
# 답변 정도를 보여주는 걸로 n=6으로 설정ㅇㅇ


In [44]:
prompts= ['점심 먹었어?', '뭐해!?','너무 졸려 근데 일이 많아...']

In [45]:
outputs: List[RequestOutput] = model.generate(prompts, sampling_params)

Processed prompts:   0%|          | 0/39 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

WARNING 11-21 19:51:00 scheduler.py:1481] Sequence group 5_parallel_sample_6 is preempted by PreemptionMode.RECOMPUTE mode because there is not enough KV cache space. This can affect the end-to-end performance. Increase gpu_memory_utilization or tensor_parallel_size to provide more KV cache memory. total_num_cumulative_preemption=1


Processed prompts:   8%|▊         | 3/39 [02:35<31:09, 51.92s/it, est. speed input: 0.15 toks/s, output: 42.93 toks/s]   


In [9]:
outputs[0].prompt

'점심 먹었어?'

In [10]:
outputs[1].prompt

'사랑해'

In [11]:
outputs[2].prompt

'너무 졸려 근데 일이 많아...'

In [12]:
outputs[0].outputs

[CompletionOutput(index=0, text='?\n아까 먹었지~\n뭐 먹었어??\n닭가슴살 먹었어~\n오 맛있었겠다~~\n응~ 닭가슴살에 양파랑 먹었어~', token_ids=(5380, 54059, 101154, 108715, 101461, 22035, 89241, 167, 99834, 108715, 101461, 32179, 30, 5380, 9019, 255, 20565, 120534, 108281, 108715, 101461, 32179, 89241, 58368, 119408, 105625, 101461, 125684, 5940, 198, 110685, 93, 35243, 255, 20565, 120534, 108281, 19954, 104870, 101508, 102581, 108715, 101461, 32179, 93, 128001), cumulative_logprob=None, logprobs=None, finish_reason=stop, stop_reason=<|end_of_text|>),
 CompletionOutput(index=1, text='?\n응응 ㅎㅎ\n오늘도 비와서\n오늘은\n어디로 가는거야?\n오늘은\n#@이름', token_ids=(5380, 110685, 110685, 103667, 236, 116507, 198, 58368, 105622, 49085, 75086, 81673, 27796, 198, 58368, 105622, 34804, 198, 32179, 90335, 17835, 36609, 16969, 93292, 90759, 5380, 58368, 105622, 34804, 198, 83172, 13094, 64254, 2), cumulative_logprob=None, logprobs=None, finish_reason=stop, stop_reason=2),
 CompletionOutput(index=2, text='?\n아니\n배고파\n나도..\n배고파\n나도\n배고파\n나도\n나도\n나도', toke

In [39]:
outputs[0].outputs[0].text

'?\n아까 먹었지~\n뭐 먹었어??\n닭가슴살 먹었어~\n오 맛있었겠다~~\n응~ 닭가슴살에 양파랑 먹었어~'

In [46]:
def extract_text_between_newlines(text):
    """
    주어진 문자열에서 첫 번째 \n과 두 번째 \n 사이의 텍스트를 추출합니다.
    \n이 없을 경우 전체 텍스트를 반환합니다.

    Args:
        text (str): 입력 문자열.

    Returns:
        str: 첫 번째와 두 번째 \n 사이의 텍스트, 또는 \n이 없으면 입력 텍스트 그대로 반환.
    """
    # 첫 번째 \n 위치 찾기
    first_newline = text.find('\n')
    
    # \n이 없다면 입력 텍스트 그대로 반환
    if first_newline == -1:
        return text.strip()
    
    # 두 번째 \n 위치 찾기
    second_newline = text.find('\n', first_newline + 1)
    
    # 두 번째 \n이 존재하면 텍스트 추출, 없으면 첫 번째 \n 이후 텍스트 반환
    if second_newline != -1:
        return text[first_newline + 1:second_newline].strip()
    else:
        return text[first_newline + 1:].strip()

# 테스트
result = extract_text_between_newlines(outputs[0].outputs[0].text)
print(result)  # "아까 먹었지~" 출력

웅웅!!


In [48]:
for i in range(3):
    # print(f'==================={i}번째 답변====================================')
    # print(extract_text_between_newlines(outputs[i].outputs[0].text))
    for j in range(6):
            print(f'==================={i,j}번째 답변====================================')
            print(extract_text_between_newlines(outputs[i].outputs[j].text))

===================(0, 0)번째 답변====================================
웅웅!!
===================(0, 1)번째 답변====================================
응응 먹었어
===================(0, 2)번째 답변====================================
아니용
===================(0, 3)번째 답변====================================
응응
===================(0, 4)번째 답변====================================
웅웅
===================(0, 5)번째 답변====================================
아니 아직 ㅎㅎ
===================(1, 0)번째 답변====================================
아까 밥먹구
===================(1, 1)번째 답변====================================
나
===================(1, 2)번째 답변====================================
나 밥먹었어 ㅎㅎ
===================(1, 3)번째 답변====================================
나 지금
===================(1, 4)번째 답변====================================
나
===================(1, 5)번째 답변====================================
ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
===================(2, 0)번째 답변====================================
ㅠㅠㅠ
===================(2, 1)번째 답변===========================